In [2]:
import findspark

In [3]:
findspark.init()

In [4]:
import pyspark

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.sql import SparkSession

In [7]:
#Create spark session
appName = "Sentiment Analysis in Spark"
spark = SparkSession \
.builder \
.appName(appName) \
.config("spark.some.config.option","some-value") \
.getOrCreate()

In [8]:
#Read Data File into Spark DataFrame
#read csv file into dataframe with automatically inferred schema
tweets_csv = spark.read.csv('C:/Users/aayushi srivastava/Documents/AayushiSrivastavaJobSearch/PySparkProjects/dataset/tweets.csv',inferSchema=True,header=True)
tweets_csv.show(truncate=False, n=3)

+------+---------+---------------+---------------------------------+
|ItemID|Sentiment|SentimentSource|SentimentText                    |
+------+---------+---------------+---------------------------------+
|1038  |1        |Sentiment140   |that film is fantastic #brilliant|
|1804  |1        |Sentiment140   |this music is really bad #myband |
|1693  |0        |Sentiment140   |winter is terrible #thumbs-down  |
+------+---------+---------------+---------------------------------+
only showing top 3 rows



In [9]:
#Select the related data
#select only "Sentiment Text" and "Sentiment" column and cast "Sentiment" column data into integer
data = tweets_csv.select("SentimentText", col("Sentiment").cast("Int").alias("label"))
data.show(truncate = False, n=5)


+---------------------------------+-----+
|SentimentText                    |label|
+---------------------------------+-----+
|that film is fantastic #brilliant|1    |
|this music is really bad #myband |1    |
|winter is terrible #thumbs-down  |0    |
|this game is awful #nightmare    |0    |
|I love jam #loveit               |1    |
+---------------------------------+-----+
only showing top 5 rows



In [10]:
#Divide data into training and testing
#divide data 70% training and 30% testing
dividedData = data.randomSplit([0.7,0.3])
trainingData = dividedData[0]
testingData = dividedData[1]
train_rows = trainingData.count()
test_rows = testingData.count()
print("Training data rows: ",train_rows,";Testing data rows:",test_rows)

Training data rows:  1357 ;Testing data rows: 575


In [11]:
#Prepare training data
#Separate "Sentiment text" into individual words using tokenizer
tokenizer = Tokenizer(inputCol = "SentimentText",outputCol = "SentimentWords")
tokenizedTrain = tokenizer.transform(trainingData)
tokenizedTrain.show(truncate=False,n=5)

+-------------------------+-----+------------------------------+
|SentimentText            |label|SentimentWords                |
+-------------------------+-----+------------------------------+
|I adore cheese #bestever |1    |[i, adore, cheese, #bestever] |
|I adore cheese #brilliant|1    |[i, adore, cheese, #brilliant]|
|I adore cheese #favorite |1    |[i, adore, cheese, #favorite] |
|I adore cheese #loveit   |1    |[i, adore, cheese, #loveit]   |
|I adore cheese #toptastic|1    |[i, adore, cheese, #toptastic]|
+-------------------------+-----+------------------------------+
only showing top 5 rows



In [12]:
#Removing stop words(unimportant words to be features)
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(),outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False,n=5)

+-------------------------+-----+------------------------------+---------------------------+
|SentimentText            |label|SentimentWords                |MeaningfulWords            |
+-------------------------+-----+------------------------------+---------------------------+
|I adore cheese #bestever |1    |[i, adore, cheese, #bestever] |[adore, cheese, #bestever] |
|I adore cheese #brilliant|1    |[i, adore, cheese, #brilliant]|[adore, cheese, #brilliant]|
|I adore cheese #favorite |1    |[i, adore, cheese, #favorite] |[adore, cheese, #favorite] |
|I adore cheese #loveit   |1    |[i, adore, cheese, #loveit]   |[adore, cheese, #loveit]   |
|I adore cheese #toptastic|1    |[i, adore, cheese, #toptastic]|[adore, cheese, #toptastic]|
+-------------------------+-----+------------------------------+---------------------------+
only showing top 5 rows



In [14]:
#Converting word features into numerical features. In spark 2.2.1, it is implemented in Hashing TF function using Austin Appleby's Murmur Hash 3 algorithm
hashTF = HashingTF(inputCol = swr.getOutputCol(),outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select(
'label','MeaningfulWords','features')
numericTrainData.show(truncate=False,n=5)

+-----+---------------------------+-------------------------------------------+
|label|MeaningfulWords            |features                                   |
+-----+---------------------------+-------------------------------------------+
|1    |[adore, cheese, #bestever] |(262144,[1689,91011,100089],[1.0,1.0,1.0]) |
|1    |[adore, cheese, #brilliant]|(262144,[1689,45361,100089],[1.0,1.0,1.0]) |
|1    |[adore, cheese, #favorite] |(262144,[1689,100089,108624],[1.0,1.0,1.0])|
|1    |[adore, cheese, #loveit]   |(262144,[1689,100089,254974],[1.0,1.0,1.0])|
|1    |[adore, cheese, #toptastic]|(262144,[1689,42010,100089],[1.0,1.0,1.0]) |
+-----+---------------------------+-------------------------------------------+
only showing top 5 rows



In [15]:
#train our classifier model using training data
lr = LogisticRegression(labelCol = "label", featuresCol = "features", maxIter = 10, regParam = 0.01)
model = lr.fit(numericTrainData)
print("Training is done!")

Training is done!


In [16]:
#prepare testing data
tokenizedTest = tokenizer.transform(testingData)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
'Label', 'MeaningfulWords', 'features')
numericTest.show(truncate = False, n=2)

+-----+---------------------------+--------------------------------------------+
|Label|MeaningfulWords            |features                                    |
+-----+---------------------------+--------------------------------------------+
|1    |[adore, cheese, #thumbs-up]|(262144,[1689,88825,100089],[1.0,1.0,1.0])  |
|1    |[adore, coffee, #brilliant]|(262144,[45361,100089,159212],[1.0,1.0,1.0])|
+-----+---------------------------+--------------------------------------------+
only showing top 2 rows



In [17]:
#Predict testing data and calculate the accuracy model
prediction = model.transform(numericTest)
predictionFinal = prediction.select(
"MeaningfulWords", "prediction","Label")
predictionFinal.show(n=4,truncate=False)
correctPrediction = predictionFinal.filter(
predictionFinal['prediction'] == predictionFinal['Label']).count()
totalData = predictionFinal.count()
print("correct pprediction:",correctPrediction,",total Data:",totalData,",accuracy:",correctPrediction/totalData)

+---------------------------+----------+-----+
|MeaningfulWords            |prediction|Label|
+---------------------------+----------+-----+
|[adore, cheese, #thumbs-up]|1.0       |1    |
|[adore, coffee, #brilliant]|1.0       |1    |
|[adore, coffee, #loveit]   |1.0       |1    |
|[adore, jam, #brilliant]   |1.0       |1    |
+---------------------------+----------+-----+
only showing top 4 rows

correct pprediction: 565 ,total Data: 575 ,accuracy: 0.9826086956521739
